In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig

### Helper functions

In [ ]:
def save_result(path_name, file_name, result):
    """
    Save the result to a file.
    
    Args:
        path_name (str): Path to save the file
        file_name (str): Name of the file
        result (str): Content to save
    """
    file_path = os.path.join(path_name, f"{file_name}.md")
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    with open(file_path, 'w') as file:
        file.write(result)
        
def setup_environment():
    """Set up the environment variables."""
    try:
        hf_key = os.environ["LLAMA3_KEY"]
    except KeyError:
        print("Please set the environment variable LLAMA3_KEY")
        hf_key = input("Enter your HuggingFace API key: ")
    return hf_key

def load_model(hf_key):
    """Load the LLM model and tokenizer."""
    quant_config = BitsAndBytesConfig(load_in_8bit=True)
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_key)
    model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_key, quantization_config=quant_config, device_map="auto", max_length=8192)
    config = AutoConfig.from_pretrained(model_name, token=hf_key)
    
    return model, tokenizer, config

def generate_response(model, tokenizer, messages, config):
    """Generate a response from the model."""
    model_inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to('cuda')

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    generated_ids = model.generate(
        model_inputs,
        do_sample=True,
        temperature=0.1,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        max_length=4500
    )

    print(f"Tokens used: {len(model_inputs[0])} out of {config.max_position_embeddings}")

    response = generated_ids[0][model_inputs.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

## 1. Set up the model

This step includes downloading the LLM from Hugging Face and setting up the API key.

You can temporarily add your API key by using Python with os.environ["LLAMA3_KEY"]="YOUR_API_KEY"

WARNING: NEVER SHARE/PUSH YOUR API KEY.

In [ ]:
hf_key = setup_environment()
model, tokenizer, config = load_model(hf_key)

## 2. Set file directories

In [ ]:
# Set the CWD to the directory of your repository
os.chdir('/home/gunes/fair/responsible-ai-model-cards/Steps-v3-thesis')

In [ ]:
model_card_template_path = "files/templates/model-card.md"
model_card_template = open(model_card_template_path, "r").read()

### Individual case to analyze

In [ ]:
case_name = "demo"

file_path = "files/cases/" +  case_name + "/description.md"
case_description = open(file_path, "r").read()

analysis_path = "files/cases/" + case_name + "/ethics-analysis.md"
ethics_analysis = open(analysis_path, "r").read()

## 3. Start Model Card Generation

In [ ]:
messages = [
    {"role": "system", "content": "You are an expert in model cards and responsible AI. Your task is to complete the provided model card template using the model description and data given. For the Ethical and Legal Analysis section of the card, utilize the provided analysis. Base all content solely on the provided description, and only fill in the sections available in the template without adding new fields. Ensure that the final output is the filled model card, with no additional comments or remarks."},
    {"role": "user", "content": f"Template to fill: {model_card_template} Description: {case_description} Ethical and Legal Analysis: {ethics_analysis}"},
]

In [ ]:
result = generate_response(model, tokenizer, messages, config)
print(result)

In [ ]:
case_path = "files/cases/" + case_name + "/"
save_result(case_path, "model-card", result)